# Metachronal wave analysis for Paramecium

This script quantifies metachronal wave (MW) properties in Paramecium cells from timelapse microscopy data. It assumes that input timelapses are pre-registered (e.g., using StackReg in ImageJ). Before running this script, the user must manually select a straight line along the direction of metachronal wave propagation in ImageJ and save the coordinates as a .csv file. The script preprocesses the image stack (illumination correction, normalization), then generates a kymograph by averaging intensities along a sliding box perpendicular to the selected segment. The kymograph represents spatial and temporal changes in ciliary activity. The script computes the 2D autocorrelation of the kymograph using the Wiener-Khinchin theorem (via Fourier transforms), and extracts MW parameters: ciliary beat frequency (CBF), wavelength, and wave velocity, with error estimates. The analysis includes steps to optimize the selection of the MW direction by generating rotated lines around the initial selection and identifying the angle that minimizes the wavelength. The output includes visualizations (kymograph, autocorrelation, power spectral densities) and a summary of MW parameters with errors saved in CSV format.

*Note*: This code loads functions from functions.py.

The code is adjusted from the metachronal wave analysis performed by Katerina Kourkoulou: https://github.com/livingpatterns/MW_dynamics_Didinium
The code follows the approach of: https://github.com/shurlimann/stentor-cilia-autocorrelation?tab=readme-ov-file where a segment of the MW is selected manualy, parametrized and by sliding a box within which the signal is averaged a kymograph is calculated.


Input:
- Paths
*file_directory* : The path to the directory containing the tif file to be analyzed. An output folder will be created in the same location.
*file_name*      : The name of the tif file of the registered Paramecium.

- Parameters from raw data:
*pixel_size*                  : The pixel size that can be found from the raw file.
*fps*                         : The frame rate that can be found from the raw file.

- Spline parameters:
*new_spatial_resolution*      : After the manual selection of the segment of the MW to be analyzed a homogeneous distribution of points is generated with a spline. Its target resolution is set in microns with this parameter. Values should be larger than the pixel_size to avoid oversampling. **Note:** There is currently a problem with the rescaling!
*s*                           : Controls trade-off of line smoothness and close fit.
*k*                           : The degree of the spline. Should be odd number. (Recommendation is 3)

- Parameters for assistive box mask:
After the spline is calculated a box mask of desired size is generated and then slided along the spline in a perpendicular manner. Intensities inside the box are averaged to create an 1D representation of the segment and generate the kymograph.
*box_width*    :   The desired box width in pixels
*box_length*   :   The desired box length in pixels


Output:
- The code saves automatically for visual inspection: the first frame before and after normalization, the selected points of the manual ROI selection, the kymograph and the autocorrelation, the PSD for the frequency (CBF) and the wavelength and four plots for evaluating the quality of the velocity detection.
- Additionally, there are two csv files:
*output_results*:  Collects or image info, a given cell_id, main metadata and all output values with the corresponding errors. In case file names and organization does not match the expected format you might need to manually give that information at the very end of the code following the comments there.
*parameters.csv*:  The parameters chosen to run the code. 

Important note: Files are overwritten in every run!

In [ ]:
import os
import cv2
import csv
import tifffile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate, signal
from scipy.fft import fft2, fftshift, ifft2, ifftshift, fft, ifft, fftfreq
from scipy.optimize import leastsq
from skimage import filters, measure
from scipy.ndimage import median_filter
from skimage.draw import disk
from skimage.morphology import disk as disk_vel
import numpy.matlib as matlib
import scipy.ndimage as ndimage
import polarTransform
from tqdm import tqdm
from scipy.signal import find_peaks, peak_widths
from functions import *
import matplotlib as mpl
# Ensure text is saved as editable text in SVG
mpl.rcParams['svg.fonttype'] = 'none'
# Set global font to Arial
mpl.rcParams['font.family'] = 'Arial'

In [ ]:
### INPUT ---------------------------------------------------------------------
# Path to folder of registered MW time-lapse and name of the tiff file to be processed
file_directory = "W:/Users/Daphne/RESULTS/Dynamics/Live imaging p.tet WT surface cell/"
file_name = "ptetwt_40x_03.tif"

## Parameters
# from tiff file:
pixel_size = 0.23  # μm     Fast camera 40x
fps = 1000
pixel_size2 = pixel_size

#--------------------------------------------------------------------------------------------------------------------------------------------
# Following parameters not needed when taking a straight line!
# Spline parameters:
# After manual selection of a segment of the MW, the points are re-parametrized to achieve a desired resolution using a spline
new_spatial_resolution = pixel_size
s=5 # smoothing condition (controls the trade-off between closeness and smoothness of fit)
k=3 # degree of the spline (avoid even numbers, recommended is k=3)
# Parameters for assistive box mask:
# To prepare for the kymograph a box mask slides over the spline within which intensities are averages.
box_width   = 1     # (pixels) width of box used to average out the intensities   (needs to be optimized)
box_length  = 5      # (pixels) length of box used to average out the intensities  (needs to be optimized)

In [ ]:
### PRE-PROCESSING ------------------------------------------------------------
# Sets file path and makes output folder
file_path = file_directory + "\\" + file_name
special_folder = "test_before_push\\"
output_path = file_directory + "\\output_MW\\" + special_folder

if not os.path.exists(output_path):
    os.makedirs(output_path)
    
# Reading file info
image, frame_width, frame_height, num_frames = open_tif(file_path)

#----- to test the file was loaded properly: -------
print("Number of frames :", num_frames)
print("Frame width:", frame_width)
print("Frame height:", frame_height)

frame_index = 0
show_frame_no_axes(image,frame_index)
plt.savefig(output_path + "first_frame.png", dpi=300)
#---------------------------------------------------

### Signal normalization
image_illum_fix = uneven_illumination_fix(image, 31)
image_norm = img_norm_simple(image_illum_fix)

### Optional: save the normalized video
tifffile.imwrite(file_path + file_name[:-4] + 'image_fix_norm.tif', image_norm)

#------- to display: ------------------------------
frame_index = 0
show_frame_no_axes(image_norm,frame_index)
plt.savefig(output_path + "first_frame_normalized.png", dpi=300)
plt.show()
#---------------------------------------------------

##### For Paramecium: Prepare in ImageJ the points of the ciliary array to be analyzed. Plot a straight line along the cilia and save the coordinates in a csv file.

In [ ]:
# use the following code to read the predefined points:
cilia_path = file_directory + "\\ptetwt_40x_03_doublets.csv"  # path to the csv file with the selected points

with open(cilia_path, 'r', newline='') as f:
    reader = csv.reader(f)
    points_manual = np.array([[int(row[0]), int(row[1])] for row in reader])

x_manual = points_manual[:, 0]
y_manual = points_manual[:, 1]
#----- to check the file was read properly -------
plt.figure()
plt.imshow(image[0,:,:], cmap='binary')
plt.scatter(x_manual,y_manual, color ='r', s = 5)
plt.show()
#-------------------------------------------------

##### Make sure the line is straight (take the first and last points of the line) and calculate the angle of the line compared to the y-axis.

In [ ]:
# Function to calculate the angle of a line compared to the y-axis
def calculate_angle(x_start, y_start, x_end, y_end):
    delta_x = x_end - x_start
    delta_y = y_end - y_start
    angle = np.degrees(np.arctan2(delta_x, delta_y))  # Angle in degrees
    return angle

# Define the original line using the first and last points of the manually drawn line
x_start, y_start = points_manual[0]
x_end, y_end = points_manual[-1]

# Create the original line as a NumPy array
original_line = np.array([[x_start, y_start], [x_end, y_end]])

# Calculate the center of the line
x_center = (x_start + x_end) / 2
y_center = (y_start + y_end) / 2

# Calculate the angle compared to the y-axis
angle = calculate_angle(x_start, y_start, x_end, y_end)

# Plot the original line
plt.figure(figsize=(8, 8))
plt.imshow(image[0], cmap='gray')
plt.plot([x_start, x_end], [y_start, y_end], 'r-', label=f'Original Line (Angle: {angle:.2f}°)')
plt.scatter(x_center, y_center, color='blue', label='Center')
plt.legend()
plt.title('Original Line')
plt.savefig(output_path + "input_line.png", dpi=300)
plt.show()

print(f"Angle of the original line compared to the y-axis: {angle:.2f}°")
# print length of the line
length = np.sqrt((x_end - x_start)**2 + (y_end - y_start)**2)
print(f"Length of the original line: {length:.2f} pixels = {length * pixel_size:.2f} microns")

In [ ]:
# Plot the kymograph for the original
# straight line
# Step 1: Define a curve/spline along the selection and generate 1D representation of the ciliary array
# Ensure the number of points is sufficient for the spline degree
if len(x_manual) <= k:
    k = len(x_manual) - 1  # Reduce k to be less than the number of points

x_cilia, y_cilia = spline_generator(x_manual, y_manual, pixel_size2, new_spatial_resolution, s, k)
x_int, y_int, x_residue, y_residue, dx, dy = x_y_to_indices(x_cilia, y_cilia)
theta = np.arctan(dx / dy) * 180 / np.pi
box, center = assistive_box_mask(box_width, box_length)
c_intensity = apply_box_mask(image_norm, box, theta, center, box_width, box_length, x_residue, y_residue, x_int, y_int, num_frames)
c_intensity = c_intensity.iloc[:, 10:]  # OPTIONAL: to crop the first few pixels that are noisy

# Step 2: Normalize the kymograph
c_intensity_min = np.min(c_intensity)
c_intensity_max = np.max(c_intensity)
c_intensity_norm = (c_intensity - c_intensity_min) / (c_intensity_max - c_intensity_min)

# Step 3: Display the kymograph
aspect_ratio = 100  # Adjust as needed
# num_frames = c_intensity.index.max() + 1 # to plot the kymograph rescaled in sec
# num_pixels = c_intensity.columns.max() + 1 # to plot the kymograph rescaled in μm
num_frames = c_intensity.shape[0]  # Number of frames
num_pixels = c_intensity.shape[1]  # Number of pixels
new_index = pd.Index(np.arange(num_frames) / fps)
new_columns = pd.Index(np.arange(num_pixels) * new_spatial_resolution)

# Reindex the DataFrame
rescaled_c_intensity = c_intensity.copy()
rescaled_c_intensity.index = new_index
rescaled_c_intensity.columns = new_columns
rescaled_c_intensity.index.name = 'time (sec)'
rescaled_c_intensity.columns.name = 'position along the cilia (\u03BCm)'

# kymograph_display_units(rescaled_c_intensity.iloc[:], aspect_ratio)
kymograph_display_units(rescaled_c_intensity.iloc[:])
plt.savefig(output_path + "kymograph_original_line.png", dpi=300)
plt.savefig(output_path + "kymograph_original_line.svg", format='svg')
plt.show()

# Step 4: Calculate and display the autocorrelation
c = autocorrelation2D_fft_units(c_intensity_norm)
index_t = np.arange(c.shape[0]) / fps
index_x = np.arange(c.shape[1]) * new_spatial_resolution
correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (μm)'))

# visualization
autocorrelation2D_visualization_units(correlation.iloc[:])
# autocorrelation2D_visualization_units(correlation.iloc[:], aspect_ratio)
plt.savefig(output_path + "autocorrelation_original_line.png", dpi=300)
plt.show()

In [ ]:
# If the kymograph has a lot of background it confuses the autocorrelation.
# This can partially be avoided with a median filter:
aspect_ratio = 100  # Adjust as needed
# Load the kymograph image (replace this with your image)
kymograph = rescaled_c_intensity
filter_size = 50

# Apply a median filter along the x-axis to suppress vertical lines
filtered_kymograph = median_filter(kymograph, size=(filter_size, 1))  # size=(1, width of filter)
corrected_kymograph = kymograph - filtered_kymograph

# Plot original and filtered kymographs side by side
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# Original kymograph
ax1.imshow(kymograph, cmap='grey', aspect='auto')
ax1.set_title('Original Kymograph')
# Filtered kymograph
ax2.imshow(filtered_kymograph, cmap='grey', aspect='auto')
ax2.set_title('Filtered Kymograph (median, (' + str(filter_size) + ',1))')
# Corrected kymograph
ax3.imshow(corrected_kymograph, cmap='grey', aspect='auto')
ax3.set_title('Corrected Kymograph')
plt.savefig(output_path + "filtered_kymograph_original_line.png")

# Using the line below you can look closer to a certain segment of the kymograph e.g. first 100 frames:
# kymograph_display_units(corrected_kymograph.iloc[:,:], aspect_ratio)
def kymograph_display_units(c_intensity):
    """
    Plots a given kymograph with rescaled axes in μm and sec.
    c_intensity should be rescaled beforehand.
    """

    plt.figure()
    plt.imshow(c_intensity, aspect = 40, origin = 'lower', extent=[c_intensity.columns[0], c_intensity.columns[-1], c_intensity.index[0], c_intensity.index[-1]], cmap = mpl.colormaps.get_cmap('gray'))
    plt.title('Kymograph of image intensity', fontsize = 13)
    plt.grid(False)
    plt.gca().invert_yaxis()
    plt.xlabel('dx (\u03BCm)', fontsize = 10)
    plt.ylabel('dt (sec)', fontsize = 10)

    return

kymograph_display_units(corrected_kymograph.iloc[250:1250,:])
plt.savefig(output_path + "kymograph_corrected_original_line_zoom.png", dpi=300)
plt.savefig(output_path + "kymograph_corrected_original_line_zoom.svg", format='svg')
# kymograph_display_units(corrected_kymograph, aspect_ratio)
# Save corrected kymograph:
corrected_kymograph.to_pickle(output_path + "corrected_kymograph_original_line.pkl")
plt.savefig(output_path + "kymograph_corrected_original_line.png", dpi=300)
plt.savefig(output_path + "kymograph_corrected_original_line.svg", format='svg')
plt.show()

# plot autocorrelation of the corrected kymograph
c = autocorrelation2D_fft_units(corrected_kymograph)
index_t = np.arange(c.shape[0]) / fps
index_x = np.arange(c.shape[1]) * new_spatial_resolution
correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (μm)'))

def autocorrelation2D_visualization_units(c):
    """
    Displays the autocorrelation in space (μm) and time (sec).
    Aspect ratio is adjusted considering that usual fps is in the magnitude of 1000.

    """

    plt.figure()
    plt.imshow(c, aspect = 60/1, origin = 'lower', extent=[c.columns[0], c.columns[-1], c.index[0], c.index[-1]], cmap = mpl.colormaps.get_cmap('jet'))
    plt.colorbar(label='Autocorrelation Intensity')

    plt.title('2D Autocorrelation', fontsize = 13)
    plt.grid(False)
    plt.gca().invert_yaxis()
    plt.xlabel('dx (\u03BCm)', fontsize = 10)
    plt.ylabel('dt (sec)', fontsize = 10)

    return

# autocorrelation2D_visualization_units(correlation.iloc[:], aspect_ratio)
autocorrelation2D_visualization_units(correlation.iloc[:200])
plt.savefig(output_path + "autocorrelation_corrected_kymograph.png", dpi=300)
plt.savefig(output_path + "autocorrelation_corrected_kymograph.svg", format='svg')
plt.show()

In [ ]:
# print min and max value of c
print(f"Min value of autocorrelation: {np.min(c)}")
print(f"Max value of autocorrelation: {np.max(c)}")

# plot c histogram of values
plt.figure()
plt.hist(c.flatten(), bins=100, color='blue', alpha=0.7)
plt.title('Histogram of Autocorrelation Values')
plt.xlabel('Autocorrelation Value')
plt.ylabel('Frequency')
# plt.grid(True)
# plt.savefig(output_path + "autocorrelation_histogram.png", dpi=300)
# plt.savefig(output_path + "autocorrelation_histogram.svg", format='svg')
plt.show()

In [ ]:
def output_CBF_average(correlation, temporal_sampling_freq):
    """
    Calculation of the CBF using the autocorrelation pattern by averaging over all lags in space.
    Errors are estimated by the full width at half maximum (FWHM).
    """

    num_frames = correlation.shape[0]
    num_dx = correlation.shape[1]

    # For space_lag = 0
    space_lag = 0
    frequencies_0, psd_t_0 = signal.welch(correlation.iloc[:, space_lag], fs=temporal_sampling_freq, nperseg=num_frames)

    size_psd = psd_t_0.shape[0]

    # For all space lags
    frequencies_all = np.zeros((num_dx, size_psd))
    psd_t_all = np.zeros((num_dx, size_psd))

    frequencies_all[0,:] = frequencies_0
    psd_t_all[0,:] = psd_t_0

    # Loop over space_lag values from 1 to num_dx - 1
    for space_lag in range(1, num_dx):
        # Calculate power spectral density using Welch's method
        frequencies, psd_t = signal.welch(correlation.iloc[:, space_lag], fs=temporal_sampling_freq, nperseg=num_frames)


        # Store the results
        frequencies_all[space_lag, :] = frequencies
        psd_t_all[space_lag, :] = psd_t

    # Average along the rows
    average_frequencies = np.mean(frequencies_all, axis=0)
    average_psd_t = np.mean(psd_t_all, axis=0)

    # std along the rows
    std_frequencies = np.std(frequencies_all, axis=0)
    std_psd_t = np.std(psd_t_all, axis=0)

    # normalization:
    average_psd_t = average_psd_t / np.max(average_psd_t)
    std_psd_t     = std_psd_t / np.max(average_psd_t)

    # Find the peaks in the psd:
    peaks, _ = signal.find_peaks(average_psd_t)

    # identify the primary frequency that should correspond to the CBF:
    sorted_peaks = peaks[np.argsort(average_psd_t[peaks])][::-1]
    primary_peak = sorted_peaks[0] if sorted_peaks[0] < 5 else sorted_peaks[0]
    CBF = average_frequencies[primary_peak]


    # Calculate FWHM
    half_max = average_psd_t[primary_peak] / 2
    left_idx = np.where(average_psd_t[:primary_peak] <= half_max)[0]
    right_idx = np.where(average_psd_t[primary_peak:] <= half_max)[0]

    if len(left_idx) > 0 and len(right_idx) > 0:
        left_half_max = average_frequencies[left_idx[-1]]
        right_half_max = average_frequencies[primary_peak + right_idx[0]]
        fwhm = right_half_max - left_half_max
        CBF_error = fwhm / 2  # Using half of FWHM as the error estimate
    else:
        CBF_error = 0  # Default to 0 if FWHM cannot be determined

    return average_frequencies, average_psd_t,std_frequencies, std_psd_t, CBF, CBF_error


def output_wavelength_average(correlation, spatial_sampling_freq):
    """
    Calculation of the wavelength using the autocorrelation pattern by averaging over all lags in time.
    Errors are estimated by the full width at half maximum (FWHM).
    """

    N = correlation.shape[1]
    num_dt = correlation.shape[0]

    # For time_lag = 0
    time_lag = 0
    wavenumbers_0, psd_x_0 = signal.welch(correlation.iloc[time_lag, :], fs=spatial_sampling_freq, nperseg=N)

    size_psd = psd_x_0.shape[0]

    wavenumbers_all = np.zeros((num_dt, size_psd))
    psd_x_all = np.zeros((num_dt, size_psd))

    wavenumbers_all[0,:] = wavenumbers_0
    psd_x_all[0,:] = psd_x_0

    for time_lag in range(1, num_dt):
        # Calculate power spectral density using Welch's method
        wavenumbers, psd_x = signal.welch(correlation.iloc[time_lag, :], fs=spatial_sampling_freq, nperseg=N)

        # Store the results
        wavenumbers_all[time_lag, :] = wavenumbers
        psd_x_all[time_lag, :] = psd_x

    # Average along the rows
    average_wavenumbers = np.mean(wavenumbers_all, axis=0)
    average_psd_x = np.mean(psd_x_all, axis=0)

    # std along the rows
    std_wavenumbers = np.std(wavenumbers_all, axis=0)
    std_psd_x = np.std(psd_x_all, axis=0)

    # normalization:
    average_psd_x = average_psd_x / np.mean(average_psd_x)
    std_psd_x     = std_psd_x /  np.mean(average_psd_x)

    peaks, _ = signal.find_peaks(average_psd_x)
    sorted_peaks = peaks[np.argsort(average_psd_x[peaks])][::-1]
    primary_peak = sorted_peaks[1] if sorted_peaks[0] <= 0 else sorted_peaks[0]
    primary_wavenumber = average_wavenumbers[primary_peak]

    # Calculate FWHM
    half_max = average_psd_x[primary_peak] / 2
    left_idx = np.where(average_psd_x[:primary_peak] <= half_max)[0]
    right_idx = np.where(average_psd_x[primary_peak:] <= half_max)[0]

    if len(left_idx) > 0 and len(right_idx) > 0:
        left_half_max = average_wavenumbers[left_idx[-1]]
        right_half_max = average_wavenumbers[primary_peak + right_idx[0]]
        fwhm = right_half_max - left_half_max
        wavenumber_error = fwhm / 2  # Using half of FWHM as the error estimate
    else:
        wavenumber_error = 0  # Default to 0 if FWHM cannot be determined

    wavelength = 1 / primary_wavenumber
    wavelength_error = (1 / primary_wavenumber)**2 * wavenumber_error

    return average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error

In [ ]:
# CBF by averaging over all space lags:
temporal_sampling_freq = fps
frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error = output_CBF_average(correlation, temporal_sampling_freq)
# Plotting of PSD with CBF:
plot_CBF(frequencies, psd_t_norm, std_psd_t, CBF, CBF_error)
plt.savefig(output_path + "PSD_CBF.png", dpi=300)
plt.show()

# λ by averaging over all space lags:
spatial_sampling_freq = 1/pixel_size  # pixel/μm
# spatial_sampling_freq = 1 # because data is already in μm (?)
average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error = output_wavelength_average(correlation.iloc[:,:], spatial_sampling_freq)
# Plotting of PSD with λ:
plot_wavelength(average_wavenumbers, average_psd_x, std_psd_x, primary_wavenumber, wavenumber_error,wavelength, wavelength_error)
plt.savefig(output_path + "PSD_wavelength.png", dpi=300)
plt.show()

##### Generate rotated lines for testing
We can generate rotated lines around the center of the original line. We use this to find the actual MW direction. The angle that corresponds to the lowest wavelength is the actual MW direction.

In [ ]:
# Function to generate rotated lines
def generate_rotated_lines(x_start, y_start, x_end, y_end, x_center, y_center, angles):
    rotated_lines = []
    for angle in angles:
        rotation_matrix = np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle))],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle))]
        ])
        original_line = np.array([[x_start, y_start], [x_end, y_end]])
        rotated_line = (original_line - [x_center, y_center]) @ rotation_matrix.T + [x_center, y_center]
        rotated_lines.append(rotated_line)
    return rotated_lines

# Define angles for rotation
angles = [-20, -15, -10, -5, 5, 10, 15, 20]  # Adjust as needed

# Generate rotated lines
rotated_lines = generate_rotated_lines(x_start, y_start, x_end, y_end, x_center, y_center, angles)

# Plot the original and rotated lines
plt.figure(figsize=(8, 8))
plt.imshow(image[0], cmap='gray')
plt.plot([x_start, x_end], [y_start, y_end], 'r-', label='Original Line')
for i, line in enumerate(rotated_lines):
    plt.plot(line[:, 0], line[:, 1], label=f'Rotated Line {i+1} ({angles[i]}°)')
plt.scatter(x_center, y_center, color='blue', label='Center')
plt.legend()
plt.title('Original and Rotated Lines')
plt.show()

In [ ]:
# # RUN IF YOU WANT TO CHANGE THE CHOSEN PEAKS
# def output_CBF_average(correlation, temporal_sampling_freq):
#     """
#     Calculation of the CBF using the autocorrelation pattern by averaging over all lags in space.
#     Errors are estimated by the full width at half maximum (FWHM).
#     """
#
#     num_frames = correlation.shape[0]
#     num_dx = correlation.shape[1]
#
#     # For space_lag = 0
#     space_lag = 0
#     frequencies_0, psd_t_0 = signal.welch(correlation.iloc[:, space_lag], fs=temporal_sampling_freq, nperseg=num_frames)
#
#     size_psd = psd_t_0.shape[0]
#
#     # For all space lags
#     frequencies_all = np.zeros((num_dx, size_psd))
#     psd_t_all = np.zeros((num_dx, size_psd))
#
#     frequencies_all[0,:] = frequencies_0
#     psd_t_all[0,:] = psd_t_0
#
#     # Loop over space_lag values from 1 to num_dx - 1
#     for space_lag in range(1, num_dx):
#         # Calculate power spectral density using Welch's method
#         frequencies, psd_t = signal.welch(correlation.iloc[:, space_lag], fs=temporal_sampling_freq, nperseg=num_frames)
#
#
#         # Store the results
#         frequencies_all[space_lag, :] = frequencies
#         psd_t_all[space_lag, :] = psd_t
#
#     # Average along the rows
#     average_frequencies = np.mean(frequencies_all, axis=0)
#     average_psd_t = np.mean(psd_t_all, axis=0)
#
#     # std along the rows
#     std_frequencies = np.std(frequencies_all, axis=0)
#     std_psd_t = np.std(psd_t_all, axis=0)
#
#     # normalization:
#     average_psd_t = average_psd_t / np.max(average_psd_t)
#     std_psd_t     = std_psd_t / np.max(average_psd_t)
#
#     # Find the peaks in the psd:
#     peaks, _ = signal.find_peaks(average_psd_t)
#
#     # identify the primary frequency that should correspond to the CBF:
#     sorted_peaks = peaks[np.argsort(average_psd_t[peaks])][::-1]
#     primary_peak = sorted_peaks[1] if sorted_peaks[0] < 5 else sorted_peaks[0]
#     CBF = average_frequencies[primary_peak]
#
#     # Calculate FWHM
#     half_max = average_psd_t[primary_peak] / 2
#     left_idx = np.where(average_psd_t[:primary_peak] <= half_max)[0]
#     right_idx = np.where(average_psd_t[primary_peak:] <= half_max)[0]
#
#     if len(left_idx) > 0 and len(right_idx) > 0:
#         left_half_max = average_frequencies[left_idx[-1]]
#         right_half_max = average_frequencies[primary_peak + right_idx[0]]
#         fwhm = right_half_max - left_half_max
#         CBF_error = fwhm / 2  # Using half of FWHM as the error estimate
#     else:
#         CBF_error = 0  # Default to 0 if FWHM cannot be determined
#
#     return average_frequencies, average_psd_t,std_frequencies, std_psd_t, CBF, CBF_error
#
#
# def output_wavelength_average(correlation, spatial_sampling_freq):
#     """
#     Calculation of the wavelength using the autocorrelation pattern by averaging over all lags in time.
#     Errors are estimated by the full width at half maximum (FWHM).
#     """
#
#     N = correlation.shape[1]
#     num_dt = correlation.shape[0]
#
#     # For time_lag = 0
#     time_lag = 0
#     wavenumbers_0, psd_x_0 = signal.welch(correlation.iloc[time_lag, :], fs=spatial_sampling_freq, nperseg=N)
#
#     size_psd = psd_x_0.shape[0]
#
#     wavenumbers_all = np.zeros((num_dt, size_psd))
#     psd_x_all = np.zeros((num_dt, size_psd))
#
#     wavenumbers_all[0,:] = wavenumbers_0
#     psd_x_all[0,:] = psd_x_0
#
#     for time_lag in range(1, num_dt):
#         # Calculate power spectral density using Welch's method
#         wavenumbers, psd_x = signal.welch(correlation.iloc[time_lag, :], fs=spatial_sampling_freq, nperseg=N)
#
#         # Store the results
#         wavenumbers_all[time_lag, :] = wavenumbers
#         psd_x_all[time_lag, :] = psd_x
#
#     # Average along the rows
#     average_wavenumbers = np.mean(wavenumbers_all, axis=0)
#     average_psd_x = np.mean(psd_x_all, axis=0)
#
#     # std along the rows
#     std_wavenumbers = np.std(wavenumbers_all, axis=0)
#     std_psd_x = np.std(psd_x_all, axis=0)
#
#     # normalization:
#     average_psd_x = average_psd_x / np.mean(average_psd_x)
#     std_psd_x     = std_psd_x /  np.mean(average_psd_x)
#
#     peaks, _ = signal.find_peaks(average_psd_x)
#     sorted_peaks = peaks[np.argsort(average_psd_x[peaks])][::-1]
#     primary_peak = sorted_peaks[0] if sorted_peaks[0] <= 0 else sorted_peaks[0]
#     primary_wavenumber = average_wavenumbers[primary_peak]
#
#     # Calculate FWHM
#     half_max = average_psd_x[primary_peak] / 2
#     left_idx = np.where(average_psd_x[:primary_peak] <= half_max)[0]
#     right_idx = np.where(average_psd_x[primary_peak:] <= half_max)[0]
#
#     if len(left_idx) > 0 and len(right_idx) > 0:
#         left_half_max = average_wavenumbers[left_idx[-1]]
#         right_half_max = average_wavenumbers[primary_peak + right_idx[0]]
#         fwhm = right_half_max - left_half_max
#         wavenumber_error = fwhm / 2  # Using half of FWHM as the error estimate
#     else:
#         wavenumber_error = 0  # Default to 0 if FWHM cannot be determined
#
#     wavelength = 1 / primary_wavenumber
#     wavelength_error = (1 / primary_wavenumber)**2 * wavenumber_error
#
#     return average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error

In [ ]:
# Initialize lists to store PSD data
psd_cbf_data = []
psd_wavelength_data = []

# Perform the same calculations for each rotated line as was done for the original line
# Convert input lists to NumPy arrays before calling spline_generator
for i, line in enumerate(rotated_lines):
    # Step 1: Define a curve/spline along the selection and generate 1D representation of the ciliary array
    x_start_rotated, y_start_rotated = line[0]
    x_end_rotated, y_end_rotated = line[1]
    # Convert to NumPy arrays
    x_manual_rotated = np.array([x_start_rotated, x_end_rotated])
    y_manual_rotated = np.array([y_start_rotated, y_end_rotated])
    # Ensure the number of points is sufficient for the spline degree
    if len(x_manual_rotated) <= k:
        k = len(x_manual_rotated) - 1  # Reduce k to be less than the number of points

    # Call spline_generator with NumPy arrays
    x_cilia_rotated, y_cilia_rotated = spline_generator(x_manual_rotated, y_manual_rotated, pixel_size, new_spatial_resolution, s, k)
    x_int_rotated, y_int_rotated, x_residue_rotated, y_residue_rotated, dx_rotated, dy_rotated = x_y_to_indices(x_cilia_rotated, y_cilia_rotated)
    theta_rotated = np.arctan(dx_rotated / dy_rotated) * 180 / np.pi
    box_rotated, center_rotated = assistive_box_mask(box_width, box_length)
    c_intensity = apply_box_mask(image_norm, box_rotated, theta_rotated, center_rotated, box_width, box_length, x_residue_rotated, y_residue_rotated, x_int_rotated, y_int_rotated, num_frames)

    # Step 2: Normalize the kymograph
    c_intensity_min = np.min(c_intensity)
    c_intensity_max = np.max(c_intensity)
    c_intensity_norm = (c_intensity - c_intensity_min) / (c_intensity_max - c_intensity_min)

    # Step 3: Display the kymograph
    num_frames = c_intensity.shape[0]
    num_pixels = c_intensity.shape[1]
    new_index = pd.Index(np.arange(num_frames) / fps)
    new_columns = pd.Index(np.arange(num_pixels) * new_spatial_resolution)
    rescaled_c_intensity = c_intensity.copy()
    rescaled_c_intensity.index = new_index
    rescaled_c_intensity.columns = new_columns
    rescaled_c_intensity.index.name = 'time (sec)'
    rescaled_c_intensity.columns.name = 'position along the cilia (\u03BCm)'

    # Apply a median filter along the x-axis to suppress vertical lines, use the same filter size as before (on the original line)
    kymograph = rescaled_c_intensity
    # filter_size = 5
    filtered_kymograph = median_filter(kymograph, size=(filter_size, 1))  # size=(1, width of filter)
    corrected_kymograph = kymograph - filtered_kymograph
    rescaled_c_intensity = corrected_kymograph

    # kymograph_display_units(rescaled_c_intensity.iloc[:500], aspect_ratio)
    kymograph_display_units(rescaled_c_intensity.iloc[:200])
    plt.savefig(output_path + f"kymograph_rotated_line_{i+1}.png", dpi=300)
    plt.show()

    # Step 4: Calculate and display the autocorrelation
    # c = autocorrelation2D_fft_units(c_intensity_norm)
    c = autocorrelation2D_fft_units(rescaled_c_intensity)
    index_t = np.arange(c.shape[0]) / fps
    index_x = np.arange(c.shape[1]) * new_spatial_resolution
    correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (\u03BCm)'))

    # autocorrelation2D_visualization_units(correlation.iloc[:], aspect_ratio)
    # plt.savefig(output_path + f"autocorrelation_rotated_line_{i+1}.png", dpi=300)
    # plt.show()

    # Calculate PSD for CBF
    temporal_sampling_freq = fps
    frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error = output_CBF_average(correlation, temporal_sampling_freq)
    psd_cbf_data.append((frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error, f'Rotated Line {i+1} ({angles[i]}°)'))

    # Calculate PSD for Wavelength
    spatial_sampling_freq = 1 / pixel_size
    # spatial_sampling_freq = 1 # because data is already in μm (?)
    average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error = output_wavelength_average(correlation, spatial_sampling_freq)
    psd_wavelength_data.append((average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, primary_wavenumber, wavenumber_error, wavelength, wavelength_error, f'Rotated Line {i+1} ({angles[i]}°)' ))


# Add the original line to PSD data
x_start, y_start = original_line[0]
x_end, y_end = original_line[1]
x_manual = np.array([x_start, x_end])
y_manual = np.array([y_start, y_end])

x_cilia, y_cilia = spline_generator(x_manual, y_manual, pixel_size, new_spatial_resolution, s, k)
x_int, y_int, x_residue, y_residue, dx, dy = x_y_to_indices(x_cilia, y_cilia)
theta = np.arctan(dx / dy) * 180 / np.pi
box, center = assistive_box_mask(box_width, box_length)
c_intensity = apply_box_mask(image_norm, box, theta, center, box_width, box_length, x_residue, y_residue, x_int, y_int, num_frames)

c_intensity_min = np.min(c_intensity)
c_intensity_max = np.max(c_intensity)
c_intensity_norm = (c_intensity - c_intensity_min) / (c_intensity_max - c_intensity_min)

# Step 3: Display the kymograph
num_frames = c_intensity.shape[0]
num_pixels = c_intensity.shape[1]
new_index = pd.Index(np.arange(num_frames) / fps)
new_columns = pd.Index(np.arange(num_pixels) * new_spatial_resolution)
rescaled_c_intensity = c_intensity.copy()
rescaled_c_intensity.index = new_index
rescaled_c_intensity.columns = new_columns
rescaled_c_intensity.index.name = 'time (sec)'
rescaled_c_intensity.columns.name = 'position along the cilia (\u03BCm)'

# Apply a median filter along the x-axis to suppress vertical lines, use the same filter size as before (on the original line)
kymograph = rescaled_c_intensity
# filter_size = 5
filtered_kymograph = median_filter(kymograph, size=(filter_size, 1))  # size=(1, width of filter)
corrected_kymograph = kymograph - filtered_kymograph
rescaled_c_intensity = corrected_kymograph

# c = autocorrelation2D_fft_units(c_intensity_norm)
c = autocorrelation2D_fft_units(rescaled_c_intensity)
index_t = np.arange(c.shape[0]) / fps
index_x = np.arange(c.shape[1]) * new_spatial_resolution
correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (\u03BCm)'))

# Calculate PSD for CBF
temporal_sampling_freq = fps
frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error = output_CBF_average(correlation, temporal_sampling_freq)
psd_cbf_data.insert(0, (frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error, "Original"))

# Calculate PSD for Wavelength
spatial_sampling_freq = 1 / pixel_size
# spatial_sampling_freq = 1 # because data is already in μm (?)
average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error = output_wavelength_average(correlation, spatial_sampling_freq)
psd_wavelength_data.insert(0, (average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, primary_wavenumber, wavenumber_error, wavelength, wavelength_error, "Original"))

# Plot combined PSD for CBF
plt.figure(figsize=(10, 6))
for frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error, label in psd_cbf_data:
    plt.plot(frequencies, psd_t_norm, label=f"{label} (CBF: {CBF:.2f} ± {CBF_error:.2f} Hz)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power Spectral Density (CBF)")
plt.title("Combined PSD of CBF for All Lines")
plt.legend()
plt.grid(True)
plt.savefig(output_path + "Combined_PSD_CBF_with_values.png", dpi=300)
plt.show()

# Plot combined PSD for Wavelength
plt.figure(figsize=(10, 6))
for average_wavenumbers, psd_x_norm, std_wavenumbers, std_psd_x, primary_wavenumber, wavenumber_error, wavelength, wavelength_error, label in psd_wavelength_data:
    plt.plot(average_wavenumbers, psd_x_norm, label=f"{label} (λ: {wavelength:.2f} ± {wavelength_error:.2f} μm)")
plt.xlabel("Wavenumber (1/μm)")
plt.ylabel("Power Spectral Density (Wavelength)")
plt.title("Combined PSD of Wavelength for All Lines")
plt.legend()
plt.grid(True)
plt.savefig(output_path + "Combined_PSD_Wavelength_with_values.png", dpi=300)
plt.show()

In [ ]:
# Initialize lists to store the new wavelength data
new_wavelength_data = []
new_wavelength_plots = []

# Perform the same calculations for each rotated line as was done for the original line
for i, line in enumerate([original_line] + rotated_lines):
    label = "Original" if i == 0 else f"Rotated Line {i} ({angles[i-1]}°)" if i > 0 else f"Rotated Line {i}"
    print(f"Processing {label}...")

    # Step 1: Define a curve/spline along the selection and generate 1D representation of the ciliary array
    x_start, y_start = line[0]
    x_end, y_end = line[1]
    x_manual = np.array([x_start, x_end])
    y_manual = np.array([y_start, y_end])
    if len(x_manual) <= k:
        k = len(x_manual) - 1  # Reduce k to be less than the number of points
    x_cilia, y_cilia = spline_generator(x_manual, y_manual, pixel_size, new_spatial_resolution, s, k)
    x_int, y_int, x_residue, y_residue, dx, dy = x_y_to_indices(x_cilia, y_cilia)
    theta = np.arctan(dx / dy) * 180 / np.pi
    box, center = assistive_box_mask(box_width, box_length)
    c_intensity = apply_box_mask(image_norm, box, theta, center, box_width, box_length, x_residue, y_residue, x_int, y_int, num_frames)

    # Step 2: Normalize the kymograph
    c_intensity_min = np.min(c_intensity)
    c_intensity_max = np.max(c_intensity)
    c_intensity_norm = (c_intensity - c_intensity_min) / (c_intensity_max - c_intensity_min)

    # Step 3: Display the kymograph
    num_frames = c_intensity.shape[0]
    num_pixels = c_intensity.shape[1]
    new_index = pd.Index(np.arange(num_frames) / fps)
    new_columns = pd.Index(np.arange(num_pixels) * new_spatial_resolution)
    rescaled_c_intensity = c_intensity.copy()
    rescaled_c_intensity.index = new_index
    rescaled_c_intensity.columns = new_columns
    rescaled_c_intensity.index.name = 'time (sec)'
    rescaled_c_intensity.columns.name = 'position along the cilia (\u03BCm)'

    # Apply a median filter along the x-axis to suppress vertical lines, use the same filter size as before (on the original line)
    kymograph = rescaled_c_intensity
    # filter_size = 5
    filtered_kymograph = median_filter(kymograph, size=(filter_size, 1))  # size=(1, width of filter)
    corrected_kymograph = kymograph - filtered_kymograph
    rescaled_c_intensity = corrected_kymograph

    # kymograph_display_units(rescaled_c_intensity.iloc[:500], aspect_ratio)
    kymograph_display_units(rescaled_c_intensity.iloc[:200])
    plt.savefig(output_path + f"kymograph_rotated_line_{i+1}.png", dpi=300)
    plt.show()

    # Step 3: Apply a median filter and calculate autocorrelation
    filtered_kymograph = median_filter(c_intensity_norm, size=(filter_size, 1))
    corrected_kymograph = c_intensity_norm - filtered_kymograph
    c = autocorrelation2D_fft_units(corrected_kymograph)
    index_t = np.arange(c.shape[0]) / fps
    index_x = np.arange(c.shape[1]) * new_spatial_resolution
    correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (μm)'))

    autocorrelation2D_visualization_units(correlation.iloc[:100])
    plt.savefig(output_path + f"autocorrelation_rotated_line_{i+1}.png", dpi=300)
    plt.show()

    # Step 4: Find peaks and calculate wavelength
    y_location = 0  # Analyze at y=1
    # peaks_w, properties = find_peaks(correlation.iloc[y_location, :], prominence=0.1)
    peaks_w, properties = find_peaks(correlation.iloc[y_location, :], prominence=0.01)
    if len(peaks_w) == 0:
        print(f"No peaks found for {label}. Skipping...")
        continue
    elif len(peaks_w) > 1:
        prominent_peak = peaks_w[1]  # Use the second peak
    elif len(peaks_w) == 1:
        prominent_peak = peaks_w[0]  # Use the only peak

    ds_peak_1 = correlation.columns[prominent_peak]
    wavelength = ds_peak_1

    # Calculate the width at relative height 0.5
    widths_half, width_heights_half, left_ips_half, right_ips_half = peak_widths(correlation.iloc[y_location, :], [prominent_peak],rel_height=0.5)
    wavelength_error = widths_half[0] / 2  # Error as half of the width at 0.5

    # Append data to the list
    new_wavelength_data.append((label, wavelength, wavelength_error, peaks_w))

    # Plot the autocorrelation with the prominent peak and its widths
    plt.figure(figsize=(10, 6))
    plt.plot(correlation.columns, correlation.iloc[y_location, :], marker='o', linestyle='-', label="Autocorrelation")
    plt.plot(correlation.columns[prominent_peak], correlation.iloc[y_location, prominent_peak], "ro", label="Prominent Peak")
    plt.hlines(width_heights_half, correlation.columns[int(left_ips_half[0])], correlation.columns[int(right_ips_half[0])], color="green", linestyle="--", label=f"Width at 0.5: {widths_half[0]:.2f} μm")
    plt.axvline(x=ds_peak_1, ymin=0, ymax=1, color='red', linestyle='--', label=f"Wavelength: {wavelength:.2f} μm")
    plt.xlabel("dx (μm)")
    plt.ylabel("Autocorrelation")
    plt.title(f"Autocorrelation with Peak Widths - {label}")
    plt.legend()
    plt.grid(True)
    plot_path = output_path + f"autocorrelation_with_widths_{label.replace(' ', '_').lower()}.png"
    plt.savefig(plot_path, dpi=300)
    plt.show()

    # Append plot path to the list
    new_wavelength_plots.append(plot_path)

# Plot combined results for the new wavelength data
plt.figure(figsize=(10, 6))
for label, wavelength, wavelength_error, peaks in new_wavelength_data:
    plt.errorbar([label], [wavelength], yerr=[wavelength_error], fmt='o', label=f"{label} (λ: {wavelength:.2f} ± {wavelength_error:.2f} μm)")
plt.xlabel("Line")
plt.ylabel("Wavelength (μm)")
plt.title("Wavelength for All Lines")
plt.legend()
plt.grid(True)
plt.savefig(output_path + "combined_wavelength_results.png", dpi=300)
plt.show()

#### It is possible to generate parallel lines to the original line, this is both to check the robustness of the CBF and wavelength results, and to obtain the average.

In [ ]:
# Function to generate a single rotated line, choose angle = 0 to get the original imported line. Or choose the best angle
def generate_rotated_line(x_start, y_start, x_end, y_end, x_center, y_center, angle):
    # Convert angle to radians
    angle_rad = np.radians(angle)

    # Create the original line as a NumPy array
    original_line = np.array([[x_start, y_start], [x_end, y_end]])

    # Translate the line to the origin
    translated_line = original_line - np.array([x_center, y_center])

    # Create the rotation matrix
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad), np.cos(angle_rad)]
    ])

    # Rotate the line
    rotated_line = np.dot(translated_line, rotation_matrix.T)

    # Translate the line back to its original position
    rotated_line += np.array([x_center, y_center])

    return rotated_line

# Example usage
angle = 5.19  # Specify the desired angle compared to the original line. Found from previous analysis

# strat from original line
x_start, y_start = original_line[0]
x_end, y_end = original_line[1]
x_center = (x_start + x_end) / 2
y_center = (y_start + y_end) / 2

# Generate the rotated line
rotated_line = generate_rotated_line(x_start, y_start, x_end, y_end, x_center, y_center, angle)

# Calculate the angle of the rotated line with the y-axis
rotated_x_start, rotated_y_start = rotated_line[0]
rotated_x_end, rotated_y_end = rotated_line[1]
rotated_angle = calculate_angle(rotated_x_start, rotated_y_start, rotated_x_end, rotated_y_end)

# Plot the original and rotated line
plt.figure(figsize=(8, 8))
plt.imshow(image[0], cmap='gray')
plt.plot([x_start, x_end], [y_start, y_end], 'r-', label=f'Original Line')
plt.plot(rotated_line[:, 0], rotated_line[:, 1], 'g-', label=f'Rotated Line (Angle with Y-axis: {rotated_angle:.0f}°)')
# plt.scatter(x_center, y_center, color='blue', label='Center')
plt.legend()
plt.title('Original and Rotated Line')
plt.show()

print(f"Angle of the rotated line compared to the y-axis: {rotated_angle:.2f}°")

In [ ]:
# Function to generate parallel lines
def generate_parallel_lines(x_start, y_start, x_end, y_end, distances):
    parallel_lines = []
    # Calculate the slope and perpendicular vector
    dx = x_end - x_start
    dy = y_end - y_start
    length = np.sqrt(dx**2 + dy**2)
    perp_vector = np.array([-dy, dx]) / length  # Unit perpendicular vector

    for distance in distances:
        offset = distance * perp_vector
        parallel_line = np.array([[x_start, y_start], [x_end, y_end]]) + offset
        parallel_lines.append(parallel_line)
    return parallel_lines

# Define distances for parallel lines
distances = [-10, -5, 5, 10]  # Adjust as needed

# Generate parallel lines
# parallel_lines = generate_parallel_lines(x_start, y_start, x_end, y_end, distances) # from original line
parallel_lines = generate_parallel_lines(rotated_x_start, rotated_y_start, rotated_x_end, rotated_y_end, distances) # from rotated line

# Plot the original and parallel lines
plt.figure(figsize=(8, 8))
plt.imshow(image[0], cmap='gray')
# plt.plot([x_start, x_end], [y_start, y_end], 'r-', label='Original Line')
plt.plot([rotated_x_start, rotated_x_end], [rotated_y_start, rotated_y_end], 'r-', label=f'Rotated Line (Angle with Y-axis: {rotated_angle:.0f}°)')
for i, line in enumerate(parallel_lines):
    plt.plot(line[:, 0], line[:, 1], label=f'Parallel Line {i+1} (Distance: {distances[i]} px)')
plt.scatter(x_center, y_center, color='blue', label='Center')
plt.legend()
plt.title('Original and Parallel Lines')
plt.savefig(output_path + "parallel_lines_usedforanalysis.png", dpi=300)
plt.show()

In [ ]:
# # OPTIONAL !!!! --> if used than change the function in the next cell to add the threshold!
# # If the frequency peak is the wrong one, you can try to change the primary peak manually by adding a threshold
#
# def output_CBF_average(correlation, temporal_sampling_freq, frequency_threshold):
#     """
#     Calculation of the CBF using the autocorrelation pattern by averaging over all lags in space.
#     Errors are estimated by the full width at half maximum (FWHM).
#     """
#     num_frames = correlation.shape[0]
#     num_dx = correlation.shape[1]
#
#     # For space_lag = 0
#     space_lag = 0
#     frequencies_0, psd_t_0 = signal.welch(correlation.iloc[:, space_lag], fs=temporal_sampling_freq, nperseg=num_frames)
#
#     size_psd = psd_t_0.shape[0]
#
#     # For all space lags
#     frequencies_all = np.zeros((num_dx, size_psd))
#     psd_t_all = np.zeros((num_dx, size_psd))
#
#     frequencies_all[0, :] = frequencies_0
#     psd_t_all[0, :] = psd_t_0
#
#     # Loop over space_lag values from 1 to num_dx - 1
#     for space_lag in range(1, num_dx):
#         frequencies, psd_t = signal.welch(correlation.iloc[:, space_lag], fs=temporal_sampling_freq, nperseg=num_frames)
#         frequencies_all[space_lag, :] = frequencies
#         psd_t_all[space_lag, :] = psd_t
#
#     # Average along the rows
#     average_frequencies = np.mean(frequencies_all, axis=0)
#     average_psd_t = np.mean(psd_t_all, axis=0)
#
#     # std along the rows
#     std_frequencies = np.std(frequencies_all, axis=0)
#     std_psd_t = np.std(psd_t_all, axis=0)
#
#     # normalization:
#     average_psd_t = average_psd_t / np.max(average_psd_t)
#     std_psd_t = std_psd_t / np.max(average_psd_t)
#
#     # Find the peaks in the psd:
#     peaks, _ = signal.find_peaks(average_psd_t)
#
#     # Filter peaks based on the x-axis (frequency threshold)
#     peaks = np.array([peak for peak in peaks if average_frequencies[peak] < frequency_threshold])
#
#     # identify the primary frequency that should correspond to the CBF:
#     sorted_peaks = peaks[np.argsort(average_psd_t[peaks])][::-1]
#     primary_peak = sorted_peaks[0] if sorted_peaks[0] < 5 else sorted_peaks[0]
#     CBF = average_frequencies[primary_peak]
#
#     # Calculate FWHM
#     half_max = average_psd_t[primary_peak] / 2
#     left_idx = np.where(average_psd_t[:primary_peak] <= half_max)[0]
#     right_idx = np.where(average_psd_t[primary_peak:] <= half_max)[0]
#
#     if len(left_idx) > 0 and len(right_idx) > 0:
#         left_half_max = average_frequencies[left_idx[-1]]
#         right_half_max = average_frequencies[primary_peak + right_idx[0]]
#         fwhm = right_half_max - left_half_max
#         CBF_error = fwhm / 2  # Using half of FWHM as the error estimate
#     else:
#         CBF_error = 0  # Default to 0 if FWHM cannot be determined
#
#     return average_frequencies, average_psd_t,std_frequencies, std_psd_t, CBF, CBF_error

In [ ]:
# Initialize lists to store PSD data
psd_cbf_data = []
psd_wavelength_data = []

# Perform the same calculations for each rotated line as was done for the original line
# Convert input lists to NumPy arrays before calling spline_generator
for i, line in enumerate(parallel_lines):
    # Step 1: Define a curve/spline along the selection and generate 1D representation of the ciliary array
    x_start, y_start = line[0]
    x_end, y_end = line[1]
    # Convert to NumPy arrays
    x_manual = np.array([x_start, x_end])
    y_manual = np.array([y_start, y_end])
    # Ensure the number of points is sufficient for the spline degree
    if len(x_manual) <= k:
        k = len(x_manual) - 1  # Reduce k to be less than the number of points

    # Call spline_generator with NumPy arrays
    x_cilia, y_cilia = spline_generator(x_manual, y_manual, pixel_size, new_spatial_resolution, s, k)
    x_int, y_int, x_residue, y_residue, dx, dy = x_y_to_indices(x_cilia, y_cilia)
    theta = np.arctan(dx / dy) * 180 / np.pi
    box, center = assistive_box_mask(box_width, box_length)
    c_intensity = apply_box_mask(image_norm, box, theta, center, box_width, box_length, x_residue, y_residue, x_int, y_int, num_frames)

    # Step 2: Normalize the kymograph
    c_intensity_min = np.min(c_intensity)
    c_intensity_max = np.max(c_intensity)
    c_intensity_norm = (c_intensity - c_intensity_min) / (c_intensity_max - c_intensity_min)

    # Step 3: Display the kymograph
    num_frames = c_intensity.shape[0]
    num_pixels = c_intensity.shape[1]
    new_index = pd.Index(np.arange(num_frames) / fps)
    new_columns = pd.Index(np.arange(num_pixels) * new_spatial_resolution)
    rescaled_c_intensity = c_intensity.copy()
    rescaled_c_intensity.index = new_index
    rescaled_c_intensity.columns = new_columns
    rescaled_c_intensity.index.name = 'time (sec)'
    rescaled_c_intensity.columns.name = 'position along the cilia (\u03BCm)'

    # Apply a median filter along the x-axis to suppress vertical lines, use the same filter size as before (on the original line)
    kymograph = rescaled_c_intensity
    # filter_size = 5
    filtered_kymograph = median_filter(kymograph, size=(filter_size, 1))  # size=(1, width of filter)
    corrected_kymograph = kymograph - filtered_kymograph
    rescaled_c_intensity = corrected_kymograph

    # kymograph_display_units(rescaled_c_intensity.iloc[:500], aspect_ratio)
    kymograph_display_units(rescaled_c_intensity.iloc[:200])
    plt.savefig(output_path + f"kymograph_parallel_line_{i+1}.png", dpi=300)
    plt.show()

    # Step 4: Calculate and display the autocorrelation
    # c = autocorrelation2D_fft_units(c_intensity_norm)
    c = autocorrelation2D_fft_units(rescaled_c_intensity)
    index_t = np.arange(c.shape[0]) / fps
    index_x = np.arange(c.shape[1]) * new_spatial_resolution
    correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (\u03BCm)'))

    # autocorrelation2D_visualization_units(correlation.iloc[:], aspect_ratio)
    # plt.savefig(output_path + f"autocorrelation_rotated_line_{i+1}.png", dpi=300)
    # plt.show()

    # Calculate PSD for CBF
    temporal_sampling_freq = fps
    # frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error = output_CBF_average(correlation, temporal_sampling_freq)
    #OPTIONAL --> only use the next 2 lines instead of the one above when you ran the previous cell and when there is a threshold needed...
    frequency_threshold = 50  # Set the threshold on the x-axis (frequency) --> only lookad a freq that are lower than this
    frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error = output_CBF_average(correlation, temporal_sampling_freq, frequency_threshold)

    psd_cbf_data.append((frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error, f'Parallel Line {i+1} ({distances[i]} px)'))

    # Calculate PSD for Wavelength
    spatial_sampling_freq = 1 / pixel_size
    # spatial_sampling_freq = 1 # because data is already in μm (?)
    average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error = output_wavelength_average(correlation, spatial_sampling_freq)
    psd_wavelength_data.append((average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, primary_wavenumber, wavenumber_error, wavelength, wavelength_error, f'Parallel Line {i+1} ({distances[i]} px)' ))


# Add the original line to PSD data
x_start, y_start = rotated_line[0]
x_end, y_end = rotated_line[1]
x_manual = np.array([x_start, x_end])
y_manual = np.array([y_start, y_end])

x_cilia, y_cilia = spline_generator(x_manual, y_manual, pixel_size, new_spatial_resolution, s, k)
x_int, y_int, x_residue, y_residue, dx, dy = x_y_to_indices(x_cilia, y_cilia)
theta = np.arctan(dx / dy) * 180 / np.pi
box, center = assistive_box_mask(box_width, box_length)
c_intensity = apply_box_mask(image_norm, box, theta, center, box_width, box_length, x_residue, y_residue, x_int, y_int, num_frames)

c_intensity_min = np.min(c_intensity)
c_intensity_max = np.max(c_intensity)
c_intensity_norm = (c_intensity - c_intensity_min) / (c_intensity_max - c_intensity_min)

# Step 3: Display the kymograph
num_frames = c_intensity.shape[0]
num_pixels = c_intensity.shape[1]
new_index = pd.Index(np.arange(num_frames) / fps)
new_columns = pd.Index(np.arange(num_pixels) * new_spatial_resolution)
rescaled_c_intensity = c_intensity.copy()
rescaled_c_intensity.index = new_index
rescaled_c_intensity.columns = new_columns
rescaled_c_intensity.index.name = 'time (sec)'
rescaled_c_intensity.columns.name = 'position along the cilia (\u03BCm)'

# Apply a median filter along the x-axis to suppress vertical lines, use the same filter size as before (on the original line)
kymograph = rescaled_c_intensity
# filter_size = 5
filtered_kymograph = median_filter(kymograph, size=(filter_size, 1))  # size=(1, width of filter)
corrected_kymograph = kymograph - filtered_kymograph
rescaled_c_intensity = corrected_kymograph

# c = autocorrelation2D_fft_units(c_intensity_norm)
c = autocorrelation2D_fft_units(rescaled_c_intensity)
index_t = np.arange(c.shape[0]) / fps
index_x = np.arange(c.shape[1]) * new_spatial_resolution
correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (\u03BCm)'))

# Calculate PSD for CBF
temporal_sampling_freq = fps
# frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error = output_CBF_average(correlation, temporal_sampling_freq)
#OPTIONAL
frequency_threshold = 50  # Set the threshold on the x-axis (frequency)
frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error = output_CBF_average(correlation, temporal_sampling_freq, frequency_threshold)

psd_cbf_data.insert(0, (frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error, "Original line"))

# Calculate PSD for Wavelength
spatial_sampling_freq = 1 / pixel_size
# spatial_sampling_freq = 1 # because data is already in μm (?)
average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error = output_wavelength_average(correlation, spatial_sampling_freq)
psd_wavelength_data.insert(0, (average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, primary_wavenumber, wavenumber_error, wavelength, wavelength_error, "Original"))

# Plot combined PSD for CBF
plt.figure(figsize=(10, 6))
for frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error, label in psd_cbf_data:
    plt.plot(frequencies, psd_t_norm, label=f"{label} (CBF: {CBF:.2f} ± {CBF_error:.2f} Hz)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power Spectral Density (CBF)")
plt.title("Combined PSD of CBF for All Lines")
plt.legend()
plt.grid(True)
plt.savefig(output_path + "Combined_PSD_CBF_with_values_parallel_lines.png", dpi=300)
plt.show()

# Plot combined PSD for Wavelength
plt.figure(figsize=(10, 6))
for average_wavenumbers, psd_x_norm, std_wavenumbers, std_psd_x, primary_wavenumber, wavenumber_error, wavelength, wavelength_error, label in psd_wavelength_data:
    plt.plot(average_wavenumbers, psd_x_norm, label=f"{label} (λ: {wavelength:.2f} ± {wavelength_error:.2f} μm)")
plt.xlabel("Wavenumber (1/μm)")
plt.ylabel("Power Spectral Density (Wavelength)")
plt.title("Combined PSD of Wavelength for All Lines")
plt.legend()
plt.grid(True)
# plt.savefig(output_path + "Combined_PSD_Wavelength_with_values_paralllel_lines.png", dpi=300)
plt.show()

In [ ]:
#IF NEEDED CHANGE FILTER SIZE KYMOGRAPH
filter_size = 50

# Initialize lists to store the new wavelength data
new_wavelength_data = []
new_wavelength_plots = []

# Perform the same calculations for each rotated line as was done for the original line
for i, line in enumerate([rotated_line] + parallel_lines):
    label = "Original line" if i == 0 else f"Parallel Line {i} ({distances[i-1]} px)" if i > 0 else f"Rotated Line {i}"
    print(f"Processing {label}...")

    # Step 1: Define a curve/spline along the selection and generate 1D representation of the ciliary array
    x_start, y_start = line[0]
    x_end, y_end = line[1]
    x_manual = np.array([x_start, x_end])
    y_manual = np.array([y_start, y_end])
    if len(x_manual) <= k:
        k = len(x_manual) - 1  # Reduce k to be less than the number of points
    x_cilia, y_cilia = spline_generator(x_manual, y_manual, pixel_size, new_spatial_resolution, s, k)
    x_int, y_int, x_residue, y_residue, dx, dy = x_y_to_indices(x_cilia, y_cilia)
    theta = np.arctan(dx / dy) * 180 / np.pi
    box, center = assistive_box_mask(box_width, box_length)
    c_intensity = apply_box_mask(image_norm, box, theta, center, box_width, box_length, x_residue, y_residue, x_int, y_int, num_frames)

    # Step 2: Normalize the kymograph
    c_intensity_min = np.min(c_intensity)
    c_intensity_max = np.max(c_intensity)
    c_intensity_norm = (c_intensity - c_intensity_min) / (c_intensity_max - c_intensity_min)

    # Step 3: Display the kymograph
    num_frames = c_intensity.shape[0]
    num_pixels = c_intensity.shape[1]
    new_index = pd.Index(np.arange(num_frames) / fps)
    new_columns = pd.Index(np.arange(num_pixels) * new_spatial_resolution)
    rescaled_c_intensity = c_intensity.copy()
    rescaled_c_intensity.index = new_index
    rescaled_c_intensity.columns = new_columns
    rescaled_c_intensity.index.name = 'time (sec)'
    rescaled_c_intensity.columns.name = 'position along the cilia (\u03BCm)'

    # Apply a median filter along the x-axis to suppress vertical lines, use the same filter size as before (on the original line)
    kymograph = rescaled_c_intensity
    # filter_size = 5
    filtered_kymograph = median_filter(kymograph, size=(filter_size, 1))  # size=(1, width of filter)
    corrected_kymograph = kymograph - filtered_kymograph
    rescaled_c_intensity = corrected_kymograph

    # kymograph_display_units(rescaled_c_intensity.iloc[:500], aspect_ratio)
    kymograph_display_units(rescaled_c_intensity.iloc[:200])
    plt.savefig(output_path + f"kymograph_parallel_line_{i+1}.png", dpi=300)
    plt.show()

    # Step 3: Apply a median filter and calculate autocorrelation
    filtered_kymograph = median_filter(c_intensity_norm, size=(filter_size, 1))
    corrected_kymograph = c_intensity_norm - filtered_kymograph
    c = autocorrelation2D_fft_units(corrected_kymograph)
    index_t = np.arange(c.shape[0]) / fps
    index_x = np.arange(c.shape[1]) * new_spatial_resolution
    correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (μm)'))

    autocorrelation2D_visualization_units(correlation.iloc[:100])
    plt.savefig(output_path + f"autocorrelation_parallel_line_{i+1}.png", dpi=300)
    plt.show()

    # Step 4: Find peaks and calculate wavelength
    y_location = 1  # Analyze at y=1
    # peaks_w, properties = find_peaks(correlation.iloc[y_location, :], prominence=0.008)
    peaks_w, properties = find_peaks(correlation.iloc[y_location, :], prominence=0.0005)
    peaks_w = [peak for peak in peaks_w if correlation.columns[peak] > 8 and correlation.columns[peak] < 11]  # Filter peaks based on wavelength range
    if len(peaks_w) == 0:
        print(f"No peaks found for {label}. Skipping...")
        continue
    elif len(peaks_w) > 1:
        prominent_peak = peaks_w[1]  # Use the second peak
    elif len(peaks_w) == 2:
        prominent_peak = peaks_w[2]  # Use the second peak
    elif len(peaks_w) == 1:
        prominent_peak = peaks_w[0]  # Use the only peak

    ds_peak_1 = correlation.columns[prominent_peak]
    wavelength = ds_peak_1

    # Calculate the width at relative height 0.5
    widths_half, width_heights_half, left_ips_half, right_ips_half = peak_widths(correlation.iloc[y_location, :], [prominent_peak],rel_height=0.5)
    wavelength_error = widths_half[0] / 2  # Error as half of the width at 0.5
    wavelength_error = wavelength_error * pixel_size  # Convert to μm
    widths_half = widths_half * pixel_size  # Convert to μm

    # Append data to the list
    new_wavelength_data.append((label, wavelength, wavelength_error, peaks_w))

    # Plot the autocorrelation with the prominent peak and its widths
    plt.figure(figsize=(10, 6))
    plt.plot(correlation.columns, correlation.iloc[y_location, :], marker='o', linestyle='-', label="Autocorrelation")
    plt.plot(correlation.columns[prominent_peak], correlation.iloc[y_location, prominent_peak], "ro", label="Prominent Peak")
    plt.hlines(width_heights_half, correlation.columns[int(left_ips_half[0])], correlation.columns[int(right_ips_half[0])], color="green", linestyle="--", label=f"Width at 0.5: {widths_half[0]:.2f} μm")
    plt.axvline(x=ds_peak_1, ymin=0, ymax=1, color='red', linestyle='--', label=f"Wavelength: {wavelength:.2f} μm")
    plt.xlabel("dx (μm)")
    plt.ylabel("Autocorrelation")
    plt.title(f"Autocorrelation with Peak Widths - {label}")
    plt.legend()
    plt.grid(True)
    plot_path = output_path + f"autocorrelation_with_widths_{label.replace(' ', '_').lower()}.png"
    plt.savefig(plot_path, dpi=300)
    plt.show()

    # Append plot path to the list
    new_wavelength_plots.append(plot_path)

# Plot combined results for the new wavelength data
plt.figure(figsize=(10, 6))
for label, wavelength, wavelength_error, peaks in new_wavelength_data:
    plt.errorbar([label], [wavelength], yerr=[wavelength_error], fmt='o', label=f"{label} (λ: {wavelength:.2f} ± {wavelength_error:.2f} μm)")
plt.xlabel("Line")
plt.ylabel("Wavelength (μm)")
plt.title("Wavelength for All Lines")
plt.legend()
plt.grid(True)
plt.savefig(output_path + "combined_wavelength_results_parallel.png", dpi=300)
plt.show()

In [ ]:
# If the results of the parallel lines are good, save the results in a dataframe and the save as csv file
wavelength_df = pd.DataFrame(new_wavelength_data, columns=['Line', 'Wavelength (μm)', 'Wavelength Error (μm)', 'Peaks'])
wavelength_df['Wavelength (μm)'] = wavelength_df['Wavelength (μm)'].astype(float)
wavelength_df['Wavelength Error (μm)'] = wavelength_df['Wavelength Error (μm)'].astype(float)
wavelength_df['Peaks'] = wavelength_df['Peaks'].apply(lambda x: ', '.join(map(str, x)))
# wavelength_df.to_csv(output_path + "wavelength_results_parallel.csv", index=False)


# Create an empty DataFrame to store the results for the CBF
cbf_results = pd.DataFrame(columns=["Line", "CBF (Hz)", "CBF Error (Hz)"])
# Iterate through the psd_cbf_data list and extract the relevant values
for _, _, _, _, CBF, CBF_error, label in psd_cbf_data:
    cbf_results = pd.concat([cbf_results, pd.DataFrame({"Line": [label], "CBF (Hz)": [CBF], "CBF Error (Hz)": [CBF_error]})], ignore_index=True)
# Save the DataFrame to a CSV file
# output_csv_path = output_path + "CBF_results.csv"
# cbf_results.to_csv(output_csv_path, index=False)
# print(f"CBF results saved to {output_csv_path}")


# From wavelength_df and cbf_results add together the wavelenght and CBF data and add the wave velocity by multiplying the CBF and the wavelength
# Merge the two DataFrames on the "Line" column
merged_df = pd.merge(wavelength_df, cbf_results, on="Line", how="outer")
# Calculate the wave velocity
merged_df["Wave Velocity (μm/s)"] = merged_df["Wavelength (μm)"] * merged_df["CBF (Hz)"]
# Save the merged DataFrame to a CSV file
output_csv_path = output_path + "All_MW_results.csv"
merged_df.to_csv(output_csv_path, index=False)

# # Calculate the mean and standard error of the mean (SEM) for the wavelength and CBF
# mean_wavelength = merged_df["Wavelength (μm)"].mean()
# std_wavelength = merged_df["Wavelength (μm)"].std()
# sem_wavelength = std_wavelength / np.sqrt(len(merged_df["Wavelength (μm)"]))
# mean_cbf = merged_df["CBF (Hz)"].mean()
# std_cbf = merged_df["CBF (Hz)"].std()
# sem_cbf = std_cbf / np.sqrt(len(merged_df["CBF (Hz)"]))
# # Save the mean and SEM to a CSV file
# mean_sem_df = pd.DataFrame({
#     "Mean Wavelength (μm)": [mean_wavelength],
#     "SEM Wavelength (μm)": [sem_wavelength],
#     "Mean CBF (Hz)": [mean_cbf],
#     "SEM CBF (Hz)": [sem_cbf]
# })
# mean_sem_df.to_csv(output_path + "mean_sem_results.csv", index=False)


# Create a dataframe with the parameters and save as csv file
parameters_df = pd.DataFrame({ 'File Name': file_name,
                               'Pixel Size (μm)': pixel_size,
                               'New Spatial Resolution (μm)': new_spatial_resolution,
                               'FPS': fps,
                               'Filter Size (median filter kymograph)': filter_size,
                               'Aspect Ratio': aspect_ratio,
                               'Number of Frames': num_frames,
                               'Angle of rotated line with Y-axis (degrees)': rotated_angle,
                               'Length of lines (px)': np.sqrt((x_end - x_start)**2 + (y_end - y_start)**2),
                               'Length of lines (μm)': np.sqrt((x_end - x_start)**2 + (y_end - y_start)**2) * pixel_size,
                               'Distance between parallel lines (px)': 5,
                               'Distance between parallel lines (μm)': 5 * pixel_size},
                               index=[0])
parameters_df.to_csv(output_path + "parameters.csv", index=False)


### When having found the perfectly rotated line (wavelength minimized = angle of 90 degrees with wave propagation direction), you can use the following code to get the wave properties with that rotation

In [ ]:
# Function to generate a single rotated line
def generate_rotated_line(x_start, y_start, x_end, y_end, x_center, y_center, angle):
    # Convert angle to radians
    angle_rad = np.radians(angle)

    # Create the original line as a NumPy array
    original_line = np.array([[x_start, y_start], [x_end, y_end]])

    # Translate the line to the origin
    translated_line = original_line - np.array([x_center, y_center])

    # Create the rotation matrix
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad), np.cos(angle_rad)]
    ])

    # Rotate the line
    rotated_line = np.dot(translated_line, rotation_matrix.T)

    # Translate the line back to its original position
    rotated_line += np.array([x_center, y_center])

    return rotated_line

# Example usage
angle = 0.  # Specify the desired angle compared to the original line
rotated_line = generate_rotated_line(x_start, y_start, x_end, y_end, x_center, y_center, angle)

# Calculate the angle of the rotated line with the y-axis
rotated_x_start, rotated_y_start = rotated_line[0]
rotated_x_end, rotated_y_end = rotated_line[1]
rotated_angle = calculate_angle(rotated_x_start, rotated_y_start, rotated_x_end, rotated_y_end)

# Plot the original and rotated line
plt.figure(figsize=(8, 8))
plt.imshow(image[0], cmap='gray')
plt.plot([x_start, x_end], [y_start, y_end], 'r-', label=f'Original Line')
plt.plot(rotated_line[:, 0], rotated_line[:, 1], 'g-', label=f'Rotated Line (Angle with Y-axis: {rotated_angle:.0f}°)')
plt.scatter(x_center, y_center, color='blue', label='Center')
plt.legend()
plt.title('Original and Rotated Line')
plt.show()

print(f"Angle of the rotated line compared to the y-axis: {rotated_angle:.2f}°")


In [ ]:
# Generate the kymograph for the rotated line
x_manual = rotated_line[:, 0]
y_manual = rotated_line[:, 1]
x_cilia, y_cilia = spline_generator(x_manual, y_manual, pixel_size2, new_spatial_resolution, s, k)
x_int, y_int, x_residue, y_residue, dx, dy = x_y_to_indices(x_cilia, y_cilia)
theta = np.arctan(dx / dy) * 180 / np.pi
box, center = assistive_box_mask(box_width, box_length)
c_intensity = apply_box_mask(image_norm, box, theta, center, box_width, box_length, x_residue, y_residue, x_int, y_int, num_frames)

# Normalize the kymograph
c_intensity_min = np.min(c_intensity)
c_intensity_max = np.max(c_intensity)
c_intensity_norm = (c_intensity - c_intensity_min) / (c_intensity_max - c_intensity_min)

# Rescale the kymograph
num_frames = c_intensity.shape[0]
num_pixels = c_intensity.shape[1]
new_index = pd.Index(np.arange(num_frames) / fps)
new_columns = pd.Index(np.arange(num_pixels) * new_spatial_resolution)
rescaled_c_intensity = c_intensity.copy()
rescaled_c_intensity.index = new_index
rescaled_c_intensity.columns = new_columns
rescaled_c_intensity.index.name = 'time (sec)'
rescaled_c_intensity.columns.name = 'position along the cilia (\u03BCm)'

# Plot the kymograph
aspect_ratio = 50
# kymograph_display_units(rescaled_c_intensity.iloc[:], aspect_ratio)
kymograph_display_units(rescaled_c_intensity.iloc[800:1000])
plt.savefig(output_path + "kymograph_rotated_line.png", dpi=300)
plt.show()


# Filter the kymograph
filter_size = 10
filtered_kymograph = median_filter(rescaled_c_intensity, size=(filter_size, 1))
corrected_kymograph = rescaled_c_intensity - filtered_kymograph

# Plot original, filtered, and corrected kymographs
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
ax1.imshow(rescaled_c_intensity, cmap='grey', aspect='auto')
ax1.set_title('Original Kymograph')
ax2.imshow(filtered_kymograph, cmap='grey', aspect='auto')
ax2.set_title(f'Filtered Kymograph (median, ({filter_size},1))')
ax3.imshow(corrected_kymograph, cmap='grey', aspect='auto')
ax3.set_title('Corrected Kymograph')
plt.savefig(output_path + "filtered_kymograph_rotated_line.png")
plt.show()

# Save the corrected kymograph
corrected_kymograph.to_pickle(output_path + "corrected_kymograph_rotated_line.pkl")

# visualize only a part of the corrected kymograph
fig, ax = plt.subplots(figsize=(5, 6))
ax.imshow(corrected_kymograph.iloc[800:1000, 14:30], cmap='gray', aspect='auto')
plt.show()

# Calculate and plot the autocorrelation
c = autocorrelation2D_fft_units(corrected_kymograph)
index_t = np.arange(c.shape[0]) / fps
index_x = np.arange(c.shape[1]) * new_spatial_resolution
correlation = pd.DataFrame(c, index=pd.Index(index_t, name='dt (sec)'), columns=pd.Index(index_x, name='dx (μm)'))
# autocorrelation2D_visualization_units(correlation.iloc[:], aspect_ratio)
autocorrelation2D_visualization_units(correlation.iloc[:100])
plt.savefig(output_path + "autocorrelation_rotated_line.png", dpi=300)
plt.show()

In [ ]:
# # If only a part of the distance of the kymograph is good/clear, then you can select a segment of the kymograph to display and analyze.
#
# # Display a segment of the kymograph for a specific range of distances
# distance_start = 14  # Start of the distance range (in μm)
# distance_end = 30    # End of the distance range (in μm)
#
# # Select the segment of the kymograph
# kymograph_segment = corrected_kymograph.loc[:, (corrected_kymograph.columns >= distance_start) & (corrected_kymograph.columns <= distance_end)]
#
# # kymograph_segment = corrected_kymograph.loc[:, :]
#
# # Plot the kymograph segment
# # kymograph_display_units(kymograph_segment, aspect_ratio)
# kymograph_display_units(kymograph_segment)
# plt.savefig(output_path + "kymograph_segment.png", dpi=300)
# plt.show()
#
# fig, ax = plt.subplots(figsize=(5, 6))
# ax.imshow(kymograph_segment.iloc[800:1000, :], cmap='gray', aspect='auto')
# plt.show()
#
# # Save the kymograph segment
# kymograph_segment.to_pickle(output_path + "kymograph_segment.pkl")
#
# # Calculate and plot the autocorrelation of the kymograph segment
# c_segment = autocorrelation2D_fft_units(kymograph_segment)
# index_t_segment = np.arange(c_segment.shape[0]) / fps
# index_x_segment = np.arange(c_segment.shape[1]) * new_spatial_resolution
# correlation_segment = pd.DataFrame(c_segment, index=pd.Index(index_t_segment, name='dt (sec)'), columns=pd.Index(index_x_segment, name='dx (μm)'))
#
# # Plot the autocorrelation of the segment
# # autocorrelation2D_visualization_units(correlation_segment.iloc[:], aspect_ratio)
# autocorrelation2D_visualization_units(correlation_segment.iloc[:])
# plt.savefig(output_path + "autocorrelation_kymograph_segment.png", dpi=300)
# plt.show()
#
# fig, ax = plt.subplots(figsize=(5, 6))
# ax.imshow(correlation_segment.iloc[:, :], cmap='gray', aspect='auto')
# plt.show()
#
# # If you want to use this segment for further analysis, then run the following code:
# # kymograph = kymograph_segment
# # correlation = correlation_segment
# # c = c_segment

In [ ]:
# # RUN ONLY IF YOU WANT TO CHANGE THE CHOSEN PEAKS
# def output_CBF_average(correlation, temporal_sampling_freq):
#     """
#     Calculation of the CBF using the autocorrelation pattern by averaging over all lags in space.
#     Errors are estimated by the full width at half maximum (FWHM).
#     """
#
#     num_frames = correlation.shape[0]
#     num_dx = correlation.shape[1]
#
#     # For space_lag = 0
#     space_lag = 0
#     frequencies_0, psd_t_0 = signal.welch(correlation.iloc[:, space_lag], fs=temporal_sampling_freq, nperseg=num_frames)
#
#     size_psd = psd_t_0.shape[0]
#
#     # For all space lags
#     frequencies_all = np.zeros((num_dx, size_psd))
#     psd_t_all = np.zeros((num_dx, size_psd))
#
#     frequencies_all[0, :] = frequencies_0
#     psd_t_all[0, :] = psd_t_0
#
#     # Loop over space_lag values from 1 to num_dx - 1
#     for space_lag in range(1, num_dx):
#         # Calculate power spectral density using Welch's method
#         frequencies, psd_t = signal.welch(correlation.iloc[:, space_lag], fs=temporal_sampling_freq, nperseg=num_frames)
#
#         # Store the results
#         frequencies_all[space_lag, :] = frequencies
#         psd_t_all[space_lag, :] = psd_t
#
#     # Average along the rows
#     average_frequencies = np.mean(frequencies_all, axis=0)
#     average_psd_t = np.mean(psd_t_all, axis=0)
#
#     # std along the rows
#     std_frequencies = np.std(frequencies_all, axis=0)
#     std_psd_t = np.std(psd_t_all, axis=0)
#
#     # normalization:
#     average_psd_t = average_psd_t / np.max(average_psd_t)
#     std_psd_t = std_psd_t / np.max(average_psd_t)
#
#     # Find the peaks in the psd:
#     peaks, _ = signal.find_peaks(average_psd_t)
#
#     # identify the primary frequency that should correspond to the CBF:
#     sorted_peaks = peaks[np.argsort(average_psd_t[peaks])][::-1]
#     primary_peak = sorted_peaks[1] if sorted_peaks[0] < 5 else sorted_peaks[0]
#     CBF = average_frequencies[primary_peak]
#
#     # Calculate FWHM
#     half_max = average_psd_t[primary_peak] / 2
#     left_idx = np.where(average_psd_t[:primary_peak] <= half_max)[0]
#     right_idx = np.where(average_psd_t[primary_peak:] <= half_max)[0]
#
#     if len(left_idx) > 0 and len(right_idx) > 0:
#         left_half_max = average_frequencies[left_idx[-1]]
#         right_half_max = average_frequencies[primary_peak + right_idx[0]]
#         fwhm = right_half_max - left_half_max
#         CBF_error = fwhm / 2  # Using half of FWHM as the error estimate
#     else:
#         CBF_error = 0  # Default to 0 if FWHM cannot be determined
#
#     return average_frequencies, average_psd_t, std_frequencies, std_psd_t, CBF, CBF_error
#
#
# def output_wavelength_average(correlation, spatial_sampling_freq):
#     """
#     Calculation of the wavelength using the autocorrelation pattern by averaging over all lags in time.
#     Errors are estimated by the full width at half maximum (FWHM).
#     """
#
#     N = correlation.shape[1]
#     num_dt = correlation.shape[0]
#
#     # For time_lag = 0
#     time_lag = 0
#     wavenumbers_0, psd_x_0 = signal.welch(correlation.iloc[time_lag, :], fs=spatial_sampling_freq, nperseg=N)
#
#     size_psd = psd_x_0.shape[0]
#
#     wavenumbers_all = np.zeros((num_dt, size_psd))
#     psd_x_all = np.zeros((num_dt, size_psd))
#
#     wavenumbers_all[0, :] = wavenumbers_0
#     psd_x_all[0, :] = psd_x_0
#
#     for time_lag in range(1, num_dt):
#         # Calculate power spectral density using Welch's method
#         wavenumbers, psd_x = signal.welch(correlation.iloc[time_lag, :], fs=spatial_sampling_freq, nperseg=N)
#
#         # Store the results
#         wavenumbers_all[time_lag, :] = wavenumbers
#         psd_x_all[time_lag, :] = psd_x
#
#     # Average along the rows
#     average_wavenumbers = np.mean(wavenumbers_all, axis=0)
#     average_psd_x = np.mean(psd_x_all, axis=0)
#
#     # std along the rows
#     std_wavenumbers = np.std(wavenumbers_all, axis=0)
#     std_psd_x = np.std(psd_x_all, axis=0)
#
#     # normalization:
#     average_psd_x = average_psd_x / np.mean(average_psd_x)
#     std_psd_x = std_psd_x / np.mean(average_psd_x)
#
#     peaks, _ = signal.find_peaks(average_psd_x)
#     sorted_peaks = peaks[np.argsort(average_psd_x[peaks])][::-1]
#     primary_peak = sorted_peaks[1] if sorted_peaks[0] <= 0 else sorted_peaks[0]
#     primary_wavenumber = average_wavenumbers[primary_peak]
#
#     # Calculate FWHM
#     half_max = average_psd_x[primary_peak] / 2
#     left_idx = np.where(average_psd_x[:primary_peak] <= half_max)[0]
#     right_idx = np.where(average_psd_x[primary_peak:] <= half_max)[0]
#
#     if len(left_idx) > 0 and len(right_idx) > 0:
#         left_half_max = average_wavenumbers[left_idx[-1]]
#         right_half_max = average_wavenumbers[primary_peak + right_idx[0]]
#         fwhm = right_half_max - left_half_max
#         wavenumber_error = fwhm / 2  # Using half of FWHM as the error estimate
#     else:
#         wavenumber_error = 0  # Default to 0 if FWHM cannot be determined
#
#     wavelength = 1 / primary_wavenumber
#     wavelength_error = (1 / primary_wavenumber) ** 2 * wavenumber_error
#
#     return average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error


In [ ]:
# CBF by averaging over all space lags:
temporal_sampling_freq = fps
frequencies, psd_t_norm, std_frequencies, std_psd_t, CBF, CBF_error = output_CBF_average(correlation, temporal_sampling_freq)

# Plotting of PSD with CBF:
plot_CBF(frequencies, psd_t_norm, std_psd_t, CBF, CBF_error)
plt.savefig(output_path + "PSD_CBF.png", dpi=300)
plt.show()

# λ by averaging over all space lags:
# spatial_sampling_freq = 1 / pixel_size  # pixel/μm
spatial_sampling_freq = 1 # because data is already in μm (?)
average_wavenumbers, average_psd_x, std_wavenumbers, std_psd_x, wavelength, wavelength_error, primary_wavenumber, wavenumber_error = output_wavelength_average(correlation.iloc[400:,:], spatial_sampling_freq)

# Plotting of PSD with λ:
plot_wavelength(average_wavenumbers, average_psd_x, std_psd_x, primary_wavenumber, wavenumber_error, wavelength, wavelength_error)
plt.savefig(output_path + "PSD_wavelength.png", dpi=300)
plt.show()

### Alternative ways to get the wavelength from the autocorrelation and kymograph

#### Because for the Paramecium data we cannot plot a very long line over time, we don't have many wavelenghts in the field of view to average over. This results in that the power spectral density does sometimes not give a very good result because there are not enough wavelengths visible next to each other (dx is too short, only 2 or 3 waves visible at the same time)

One method to get the wavelength for this data: look at the autocorrelation of the kymograph and find the peaks in the autocorrelation for a specific timepoint (plot autocorrelation intensity in 1D for y=0). The distance between the peaks gives the wavelength.

In [ ]:
# autocorrelation2D_visualization_units(correlation.iloc[:], aspect_ratio)
autocorrelation2D_visualization_units(correlation.iloc[:100])
plt.savefig(output_path + "autocorrelation_rotated_line.png", dpi=300)
plt.show()

# plot a part of the autocorrelation plot
fig, ax = plt.subplots(figsize=(5, 6))
ax.imshow(correlation.iloc[0:100, :], cmap='gray', aspect='auto')
plt.show()

point_y = 1 # the timepoint you want to analyze (choose y=0 or y=1 to get the most clear peaks (top of the autocorrelation plot))

# find peaks in the autocorrelation in space at timepoint y=0
peaks_w, _ = signal.find_peaks(correlation.iloc[point_y, :], prominence=0.1)
# peaks_w, _ = signal.find_peaks(correlation.iloc[0, :], prominence=0.02)
print("detected peak indices wavelength", peaks_w)
print(peaks_w[1:])
# ds_peak, = correlation.columns[peaks_w]
# print("space between nodes: ", ds_peak)
# wavelength_MW = ds_peak
# print("wavelength (\u03BCm): ", wavelength_MW)


# # Plot the autocorrelation for a specific timepoint (y=0)
# plt.figure(figsize=(10, 6))
# plt.plot(correlation.iloc[1, :], marker='o', linestyle='-')
# # plt.axvline(x=ds_peak, ymin=0, ymax=1, color='red', linestyle='--')
# plt.axvline(x=peaks_w, ymin=0, ymax=1, color='red', linestyle='--')
# # plt.axvline(x=ds_peak, ymin=0, ymax=1, color='red', linestyle='--')
# plt.xlabel('dx (μm)')
# plt.ylabel('Autocorrelation')
# plt.title('Autocorrelation at dt=0 (y=0)')
# plt.grid(True)
# plt.savefig(output_path + "autocorrelation_y0_rotated_line.png", dpi=300)
# plt.show()

# Plot the autocorrelation for a specific timepoint (y=0)
plt.figure(figsize=(10, 6))
plt.plot(correlation.iloc[point_y, :], marker='o', linestyle='-')

# Plot vertical lines for each detected peak
for peak in peaks_w:
    plt.axvline(x=correlation.columns[peak], ymin=0, ymax=1, color='red', linestyle='--')

plt.xlabel('dx (μm)')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation at dt=0 (y=0)')
plt.grid(True)
plt.savefig(output_path + "autocorrelation_y0_rotated_line.png", dpi=300)
plt.show()

Another way to get the wavelength for this data: look directly at the kymograph and find the peaks in the kymograph for a specific timepoint (plot kymograph intensity in 1D for y=0). The distance between the peaks gives the wavelength.

In [ ]:
# visualize only a part of the NOT corrected kymograph
fig, ax = plt.subplots(figsize=(5, 6))
ax.imshow(kymograph.iloc[850:900, :], cmap='gray', aspect='auto')
plt.show()

# visualize only a part of the corrected kymograph
fig, ax = plt.subplots(figsize=(5, 6))
ax.imshow(corrected_kymograph.iloc[850:900, :], cmap='gray', aspect='auto')
plt.show()

# plot the kymograph intensity for a specific timepoint in 1D (y=0)
timepoint_y = 890 # the timepoint you want to analyze

# # find peaks in the kymograph intensity in space at timepoint_y
# peaks_w, _ = signal.find_peaks(kymograph.iloc[timepoint_y, :], prominence=0.1)
# # peaks_w, _ = signal.find_peaks(kymograph.iloc[0, :], prominence=0.02)
# print("detected peak indices wavelength", peaks_w)
# print(peaks_w[1:])
# # ds_peak, = kymograph.columns[peaks_w]
# # print("space between nodes: ", ds_peak)
# # wavelength_MW = ds_peak
# # print("wavelength (\u03BCm): ", wavelength_MW)
#
# plt.figure(figsize=(10, 6))
# plt.plot(kymograph.iloc[timepoint_y, :], marker='o', linestyle='-')
# plt.axvline(x=kymograph.columns[peaks_w[1]], ymin=0, ymax=1, color='red', linestyle='--')
# plt.xlabel('dx (μm)')
# plt.ylabel('Kymograph Intensity')
# plt.title('Kymograph Intensity 1D')
# plt.show()

# Invert the kymograph intensity for the specified timepoint
inverted_data = -kymograph.iloc[timepoint_y, :]
# Find peaks in the inverted data (equivalent to finding valleys in the original data)
valleys_w, _ = signal.find_peaks(inverted_data, prominence=0.1)
# Print the detected valley indices
print("Detected valley indices (wavelength):", valleys_w)
print(valleys_w[1:])

# Plot the inverted kymograph intensity for a specific timepoint
plt.figure(figsize=(10, 6))
plt.plot(inverted_data, marker='o', linestyle='-')
# Plot vertical lines for each detected valley
for valley in valleys_w:
    plt.axvline(x=kymograph.columns[valley], ymin=0, ymax=1, color='red', linestyle='--')
plt.xlabel('dx (μm)')
plt.ylabel('Inverted Kymograph Intensity')
plt.title('Inverted Kymograph Intensity 1D')
plt.show()


#### Calculate frequency and wavelength by finding the autocorrelation and the corresponding dt and dx


In [ ]:
# Normalize the kymograph:
corrected_kymograph_min  = np.min(corrected_kymograph)
corrected_kymograph_max  = np.max(corrected_kymograph)
corrected_kymograph_norm = (corrected_kymograph - corrected_kymograph_min)/(corrected_kymograph_max - corrected_kymograph_min)

corrected_kymograph_norm.to_pickle(output_path + "corrected_kymograph_norm.pkl")

In [ ]:
def autocorrelation2D_visualization_units(c):
    """
    Displays the autocorrelation in space (μm) and time (sec).
    Aspect ratio is adjusted considering that usual fps is in the magnitude of 1000.

    """

    plt.figure()
    plt.imshow(c, aspect = 100/1, origin = 'lower', extent=[c.columns[0], c.columns[-1], c.index[0], c.index[-1]], cmap = mpl.colormaps.get_cmap('jet'))

    plt.title('2D Autocorrelation', fontsize = 13)
    plt.gca().invert_yaxis()
    plt.xlabel('dx (\u03BCm)', fontsize = 10)
    plt.ylabel('dt (sec)', fontsize = 10)

    return

In [ ]:
# Save c_intensity:
rescaled_c_intensity.to_pickle(output_path + "rescaled_c_intensity.pkl")

c = autocorrelation2D_fft_units(corrected_kymograph_norm.iloc[:200,:])
# c = autocorrelation2D_fft_units(corrected_kymograph_norm)

index_t = np.arange(c.shape[0])
index_x = np.arange(c.shape[1])
# Turning c to dataframe:
# correlation = pd.DataFrame(data = c, index = pd.Index(index_t/fps), columns = pd.Index(index_x*pixel_size))
correlation = pd.DataFrame(data = c, index = pd.Index(index_t/fps), columns = pd.Index(index_x))
correlation.index.name = 'dt (sec)'
correlation.columns.name = 'dx (\u03BCm)'

# visualization
autocorrelation2D_visualization_units(correlation)
plt.savefig(output_path + "autocorrelation_corrected.png", dpi=300)

# Save correlation:
correlation.to_pickle(output_path + "correlation_corrected.pkl")

plt.show()

In [ ]:
min_value = correlation.min().min()  # Minimum value
max_value = correlation.max().max()  # Maximum value

print(min_value)
print(max_value)

print(max_value - min_value)
correlation_normalized = (correlation - min_value) / (max_value - min_value)
# correlation_normalized

In [ ]:
# visualization
autocorrelation2D_visualization_units(correlation_normalized)
plt.show()

In [ ]:
y_location = 1  # The timepoint you want to analyze (choose y=0 or y=1 to get the most clear peaks (top of the autocorrelation plot))

# Find the most prominent peak in autocorrelation
peaks_w, properties = find_peaks(correlation_normalized.iloc[y_location, :], prominence=0.1)
prominent_peak = peaks_w[np.argmax(properties["prominences"])]  # Select the most prominent peak
print("detected peak indices wavelength", peaks_w)
print(peaks_w[1:])
ds_peak_1, = correlation_normalized.columns[[peaks_w[1]]]
print("space between nodes: ", ds_peak_1)
wavelength_MW_1 = ds_peak_1
print("wavelength 1 (\u03BCm): ", wavelength_MW_1)

# Calculate the width at relative height 0.5
widths_half, width_heights_half, left_ips_half, right_ips_half = peak_widths(
    correlation_normalized.iloc[y_location, :], [prominent_peak], rel_height=0.5
)

# Calculate the width at relative height 1.0
widths_full, width_heights_full, left_ips_full, right_ips_full = peak_widths(
    correlation_normalized.iloc[y_location, :], [prominent_peak], rel_height=1.0
)

# Print the width values
print(f"Width at 0.5 relative height: {widths_half[0]:.2f} μm")
print(f"Width at 1.0 relative height: {widths_full[0]:.2f} μm")


# Plot the autocorrelation with the prominent peak and its widths
plt.figure(figsize=(10, 6))
plt.plot(correlation_normalized.columns, correlation_normalized.iloc[1, :], marker='o', linestyle='-', label="Autocorrelation")
plt.plot(correlation_normalized.columns[prominent_peak], correlation_normalized.iloc[1, prominent_peak], "ro", label="Prominent Peak")

# Plot the contour lines for the widths
plt.hlines(
    width_heights_half, correlation_normalized.columns[int(left_ips_half[0])], correlation_normalized.columns[int(right_ips_half[0])],
    color="green", linestyle="--", label=f"Width at 0.5: {widths_half[0]:.2f} μm"
)
plt.hlines(
    width_heights_full, correlation_normalized.columns[int(left_ips_full[0])], correlation_normalized.columns[int(right_ips_full[0])],
    color="purple", linestyle="--", label=f"Width at 1.0: {widths_full[0]:.2f} μm"
)
plt.axvline(x=ds_peak_1, ymin=0, ymax=1, color='red', linestyle='--')
plt.xlabel("dx (μm)")
plt.ylabel("Autocorrelation")
plt.title("Autocorrelation in Space (dt=0) with Peak Widths")
plt.legend()
plt.grid(True)
plt.savefig(output_path + "autocorrelation_with_widths.png", dpi=300)
plt.show()


error_wavelength = widths_half[0] / 2  # Using half of the width at 0.5 relative height as the error estimate
print("Error in wavelength estimation (\u03BCm): ", error_wavelength)
measured_wavelength = ds_peak_1
print("Measured wavelength (\u03BCm): ", measured_wavelength)

# Quick velocity estimation:
# velocity_perceived_1 = ds_peak_1 / dt_peak
velocity_perceived_1 = ds_peak_1 * CBF
print("velocity perceived (\u03BCm/sec): ", velocity_perceived_1)

In [ ]:
def save_csv_file(output_path, headers, data):
    """
    Saves fileparameters or results in csv file.
    """
    with open(output_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        writer.writerow(data)
    return
x_new_cut_start = 0
x_new_cut_end   = 80
## Save parameters to csv:
headers_param = ['file_name','pixel_size', 'fps', 'x_new_cut_start', 'x_new_cut_end', 'CBF', 'CBF error', 'wavelength', 'wavelength error', 'velocity_perceived_1']
data_parameters = [file_name, pixel_size, fps, x_new_cut_start, x_new_cut_end, CBF, CBF_error, measured_wavelength, error_wavelength, velocity_perceived_1]
save_csv_file(output_path+'parameters-results.csv', headers_param, data_parameters)